In [2]:
import numpy as np
import pandas as pd

## We create our first function, which will unnest any json text into plain text

In [4]:
def unnest(path, dicc):
    """
    Read json text and convert it as plain text.
    
    Parameters
    ----------
    path : str
        path where the json file is located.
    dicc : dict
        dictionary with key-value pairs.

    returns
    -------
    DataFrame
        result of converting said dictionary to a DataFrame.

    Examples
    --------
    >>> unnest('./folder/file.csv', dictionary)
    {
    "names": ["Michael", "Jim", "Dwight"], 
    "Position": ["Manager", "Salesman", "Salesman"],
    "Age": [42, 29, 27]
    }
    """

    df = []
    with open(path, encoding = 'UTF-8-SIG') as f:
        for line in f:
            df.append(eval(line))
    
    for i in df:
        for clave, valor in i.items():
            dicc[clave].append(valor)
    dicc = pd.DataFrame(dicc)
    return dicc

In [ ]:
# We make our dictionary with key names same as the column names and an empty list.
diccionarioReviews = {'user_id' : [], 'user_url': [], 'reviews': []}

reviews = unnest('./DataSets/australian_user_reviews.json', diccionarioReviews)

In [ ]:
reviews.head()

In [ ]:
reviews.info()

We drop any duplicate value our dataset may have

In [ ]:
reviews[reviews['user_id'].duplicated(keep = False)].sort_values(by = 'user_id')

In [ ]:
reviews.drop_duplicates(subset = 'user_id', keep = False, inplace = True)

In [ ]:
reviews.head().sort_values(by = 'user_id')

We repeat this process with our items DataSet.

In [ ]:
items = {'user_id' : [], 'items_count': [], 'steam_id': [], 'user_url': [], 'items': []}
items = unnest('./DataSets/australian_users_items.json', items)


In [ ]:
items.head()

In [ ]:
items.info()

We check for any duplicate value.

In [ ]:
items[items['user_id'].duplicated(keep = False)].sort_values(by = 'user_id')

We drop said values

In [ ]:
items.drop_duplicates(subset = 'user_id', keep = 'first', inplace = True)

We make our Steam DataSet

In [ ]:
df = []

with open('./DataSets/output_steam_games.json', encoding = 'UTF-8-SIG') as f:
    for line in f:
        if '"id": NaN'in line:
            continue
        line = line.replace("true", "True")
        line = line.replace("false", "False")
        line = line.replace("NaN", "None")
        df.append(eval(line))

steam = pd.DataFrame(df)

In [ ]:
steam[steam['title'].duplicated(keep = False)].sort_values(by = 'title')

In [ ]:
steam.drop_duplicates(subset = 'id', keep = 'first', inplace = True)

We transform the 'release_date' values from str type to a Date type

In [ ]:
# Passing errors = 'coerce' will force an out-of-bounds date to NaT, in addition to forcing 
# non-dates (or non-parseable dates) to NaT.
steam['release_date'] = pd.to_datetime(steam['release_date'], format = '%Y-%m-%d', errors = 'coerce')

In [ ]:
steam['price']

We see that the price column has mixed type values, we normalise these values changing them to numbers

In [ ]:
# We do a function just in case we have multiple columns/DataSets to change.
def toZero(var):
    """
    Take parameter and change it to zero, regardless of its type.

    parameters
    ----------
    n : str
        Non numeric value.

    examples
    --------
    >>> toZero('Hello')
    0
    >>> toZero(5)
    5.0
    """
    
    try:
        return float(var)
    except:
        return 0


In [ ]:
steam['price'] = steam['price'].apply(toZero)

In [ ]:
steam.head(1)

In [ ]:
# Get rid of the unnecesary columns. This columns will not be used in our API.
steamColumns = ['publisher', 'app_name', 'url', 'reviews_url', 'early_access', 'specs', 'url', 'reviews_url']
steam.drop(steamColumns, axis = 1, inplace = True)

steam.head()

We procede to unnest the remaining list-columns into several columns.

In [ ]:
# Check the key names.
reviews['reviews'][0]

In [ ]:
reviewDF ={'user_id': [], 'funny': [],'posted': [], 'last_edited': [], 'item_id': [], 'helpful':[], 'recommend': [], 'review': [] }

for index, review in enumerate(reviews['reviews']):
    for r in list(review):
        for key, value in r.items():
            reviewDF[key].append(value)
        reviewDF['user_id'].append(reviews.iloc[index]['user_id'])

reviewDF = pd.DataFrame(reviewDF)
reviewDF.head()

Transform the text in the `posted` column into a Date value.

In [ ]:
from datetime import datetime
reviewDF['posted'] = reviewDF['posted'].str.replace('Posted ', "")
reviewDF['posted'] = pd.to_datetime(reviewDF['posted'], format = '%B%d%Y.', errors = 'ignore')

In [ ]:
reviewDF.drop(columns= ['funny', 'last_edited', 'helpful'], inplace = True)
reviewDF

Unnest the items column in the items DataFrame

In [ ]:
itemsDF = {'user_id': [], 'item_id': [], 'item_name': [], 'playtime_forever': [], 'playtime_2weeks': []}

for index, item_list in enumerate(items['items']):
    for item in item_list:
        itemsDF['user_id'].append(items.iloc[index]['user_id'])
        for key in itemsDF.keys():
            if key != 'user_id':
                itemsDF[key].append(item.get(key, None))  # Use get() to handle missing keys

itemsDF = pd.DataFrame(itemsDF)
itemsDF.head()


Apply sentiment analysis to the review column.

In [ ]:
from textblob import TextBlob

def sentiment_analysis(column):
    '''
    Read text from a DataFrame, create a numeric column corresponding to previous one.
    This new column will have one for a neutral review, zero for bad review and two for a good review.

    Parameters
    ----------
    column : pandas.core.series.Series
        column name in which the analysis will be applied

    Returns
    -------
    DataFrame
        A DataFrame with a numeric column corresponding to review

    Examples
    --------
    >>> sentiment_analysis(positiveText)
    2
    >>> sentiment_analysis(negativeText)
    0
    >>> sentiment_analysis(neutralText)
    1
    '''
    analysis = TextBlob(column)
    if analysis.sentiment.polarity < 0:
        return 0
    elif analysis.sentiment.polarity == 0:
        return 1
    else:
        return 2

Apply the function to the DataFrame

In [ ]:
reviewDF['sentiment_analysis'] = reviewDF['review'].apply(sentiment_analysis)
reviewDF['sentiment_analysis'].fillna(1, inplace = True)

Drop the review column, which will no longer be needed.

In [ ]:
reviewDF.drop('review', axis = 1, inplace = True)

In [7]:
review

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,"November 5, 2011.",1250,True,2
1,76561197970982479,"July 15, 2011.",22200,True,2
2,76561197970982479,"April 21, 2011.",43110,True,2
3,js41637,"June 24, 2014.",251610,True,2
4,js41637,"September 8, 2013.",227300,True,0
...,...,...,...,...,...
57563,76561198312638244,July 10.,70,True,2
57564,76561198312638244,July 8.,362890,True,2
57565,LydiaMorley,July 3.,273110,True,2
57566,LydiaMorley,July 20.,730,True,2


We add a 'year' column to the steam DataSet, taken from the 'release_date' column.

In [9]:
steam['year'] = pd.to_datetime(steam['release_date']).dt.year
steam

,genres,title,release_date,tags,price,id,developer,year
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,761140,Kotoshiro,2018
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",0.00,643980,Secret Level SRL,2018
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",0.00,670290,Poolians.com,2017
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017-12-07,"['Action', 'Adventure', 'Casual']",0.99,767400,彼岸领域,2017
4,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-01-04,"['Action', 'Adventure', 'Simulation', 'FPS', '...",3.99,772540,Trickjump Games Ltd,2018
...,...,...,...,...,...,...,...,...
28348,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab it Up!,2018-01-04,"['Action', 'Indie', 'Casual', 'Violent', 'Adve...",1.99,745400,Bidoniera Games,2018
28349,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",1.99,773640,"Nikita ""Ghost_RUS""",2018
28350,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,2018-01-04,"['Strategy', 'Indie', 'Casual']",4.99,733530,Sacada,2018
28351,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018-01-04,"['Indie', 'Simulation', 'Racing']",1.99,610660,Laush Dmitriy Sergeevich,2018


### We drop unnecesary columns.

In [10]:
steam.drop(columns = ['release_date', 'tags', 'price', 'developer'], inplace= True)

In [11]:
steam.head()

,genres,title,id,year
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,761140,2018
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,643980,2018
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,670290,2017
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,767400,2017
4,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,772540,2018


### Now we get rid of any null values that take space in our DataFrames

In [3]:
steam.isnull().sum()

genres          0
title           0
release_date    0
tags            0
price           0
id              0
developer       0
dtype: int64

We repeat this process for the remaining DataSets.

In [ ]:
itemsDF.isnull().sum()

In [ ]:
reviewDF.isnull().sum()

Save the DataFrames.

In [ ]:
import parquet
import pyarrow
import fastparquet
reviewDF.to_parquet('./data/review.parquet', index = False)
itemsDF.to_parquet('./data/items.parquet', index = False)
steam.to_csv('./data/steam.csv', index = False, errors = 'replace')

In [57]:
steam = pd.read_csv('./data/steam.csv', encoding = 'UTF-8')
review = pd.read_parquet('./data/review.parquet')
items = pd.read_parquet('./data/items.parquet')

## Now we need to prepare the files we are going to use for each function. We do so by merging the DataSets and keeping the columns we need.

In [60]:
steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28353 entries, 0 to 28352
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28353 non-null  object 
 1   title         28353 non-null  object 
 2   release_date  28353 non-null  object 
 3   tags          28353 non-null  object 
 4   price         28353 non-null  float64
 5   id            28353 non-null  int64  
 6   developer     28353 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.5+ MB


In [80]:
steam.head(2)

,genres,title,release_date,tags,price,id,developer
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,761140,Kotoshiro
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",0.00,643980,Secret Level SRL


In [59]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57568 entries, 0 to 57567
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             57568 non-null  object
 1   posted              57568 non-null  object
 2   item_id             57568 non-null  object
 3   recommend           57568 non-null  bool  
 4   sentiment_analysis  57568 non-null  int64 
dtypes: bool(1), int64(1), object(3)
memory usage: 1.8+ MB


In [81]:
from ast import literal_eval
steam['genres'] = steam['genres'].apply(literal_eval)
df = steam.explode('genres')

In [82]:
df

,genres,title,release_date,tags,price,id,developer
0,Action,Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,761140,Kotoshiro
0,Casual,Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,761140,Kotoshiro
0,Indie,Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,761140,Kotoshiro
0,Simulation,Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,761140,Kotoshiro
0,Strategy,Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,761140,Kotoshiro
...,...,...,...,...,...,...,...
28351,Indie,Russian Roads,2018-01-04,"['Indie', 'Simulation', 'Racing']",1.99,610660,Laush Dmitriy Sergeevich
28351,Racing,Russian Roads,2018-01-04,"['Indie', 'Simulation', 'Racing']",1.99,610660,Laush Dmitriy Sergeevich
28351,Simulation,Russian Roads,2018-01-04,"['Indie', 'Simulation', 'Racing']",1.99,610660,Laush Dmitriy Sergeevich
28352,Casual,EXIT 2 - Directions,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",4.99,658870,"xropi,stev3ns"


We change the 'item_id' column type to int64 so that we can merge both datasets on a same data type

In [118]:
items['item_id'] = items['item_id'].astype('int64')

In [119]:
merged = pd.merge(steam, items, left_on = 'id', right_on = 'item_id', how = 'inner')

In [120]:
merged.head()

,genres,title,release_date,tags,price,id,developer,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,"[Action, Indie, Racing]",Carmageddon Max Pack,1997-06-30,"['Racing', 'Action', 'Classic', 'Indie', 'Gore...",9.99,282010,Stainless Games Ltd,UTNerd24,282010,Carmageddon Max Pack,5,0
1,"[Action, Indie, Racing]",Carmageddon Max Pack,1997-06-30,"['Racing', 'Action', 'Classic', 'Indie', 'Gore...",9.99,282010,Stainless Games Ltd,I_DID_911_JUST_SAYING,282010,Carmageddon Max Pack,0,0
2,"[Action, Indie, Racing]",Carmageddon Max Pack,1997-06-30,"['Racing', 'Action', 'Classic', 'Indie', 'Gore...",9.99,282010,Stainless Games Ltd,76561197962104795,282010,Carmageddon Max Pack,0,0
3,"[Action, Indie, Racing]",Carmageddon Max Pack,1997-06-30,"['Racing', 'Action', 'Classic', 'Indie', 'Gore...",9.99,282010,Stainless Games Ltd,r3ap3r78,282010,Carmageddon Max Pack,0,0
4,"[Action, Indie, Racing]",Carmageddon Max Pack,1997-06-30,"['Racing', 'Action', 'Classic', 'Indie', 'Gore...",9.99,282010,Stainless Games Ltd,saint556,282010,Carmageddon Max Pack,13,0


In [121]:
merged['year'] = pd.to_datetime(merged['release_date']).dt.year
merged['year'] = merged['year'].astype('int64')

In [122]:
merged.head()

,genres,title,release_date,tags,price,id,developer,user_id,item_id,item_name,playtime_forever,playtime_2weeks,year
0,"[Action, Indie, Racing]",Carmageddon Max Pack,1997-06-30,"['Racing', 'Action', 'Classic', 'Indie', 'Gore...",9.99,282010,Stainless Games Ltd,UTNerd24,282010,Carmageddon Max Pack,5,0,1997
1,"[Action, Indie, Racing]",Carmageddon Max Pack,1997-06-30,"['Racing', 'Action', 'Classic', 'Indie', 'Gore...",9.99,282010,Stainless Games Ltd,I_DID_911_JUST_SAYING,282010,Carmageddon Max Pack,0,0,1997
2,"[Action, Indie, Racing]",Carmageddon Max Pack,1997-06-30,"['Racing', 'Action', 'Classic', 'Indie', 'Gore...",9.99,282010,Stainless Games Ltd,76561197962104795,282010,Carmageddon Max Pack,0,0,1997
3,"[Action, Indie, Racing]",Carmageddon Max Pack,1997-06-30,"['Racing', 'Action', 'Classic', 'Indie', 'Gore...",9.99,282010,Stainless Games Ltd,r3ap3r78,282010,Carmageddon Max Pack,0,0,1997
4,"[Action, Indie, Racing]",Carmageddon Max Pack,1997-06-30,"['Racing', 'Action', 'Classic', 'Indie', 'Gore...",9.99,282010,Stainless Games Ltd,saint556,282010,Carmageddon Max Pack,13,0,1997


Delete the unnecesary columns

In [123]:
merged.drop(['genres', 'playtime_2weeks', 'developer', 'release_date', 'price', 'item_id', 'tags', 'item_name'], axis = 1, inplace = True)

In [124]:
merged.head()

,title,id,user_id,playtime_forever,year
0,Carmageddon Max Pack,282010,UTNerd24,5,1997
1,Carmageddon Max Pack,282010,I_DID_911_JUST_SAYING,0,1997
2,Carmageddon Max Pack,282010,76561197962104795,0,1997
3,Carmageddon Max Pack,282010,r3ap3r78,0,1997
4,Carmageddon Max Pack,282010,saint556,13,1997


Now we can create a smaller file which will be more efficient in memory terms.

In [33]:
df2.to_parquet('./data/genre_functions.parquet', index = False)

In [39]:
df = pd.read_parquet('./data/genre_functions.parquet')

### Now we proceed creating a DataSet for our user system recommendation.

In [125]:
merged.head(1)

,title,id,user_id,playtime_forever,year
0,Carmageddon Max Pack,282010,UTNerd24,5,1997


In [126]:
review.head(1)

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,"November 5, 2011.",1250,True,2


In [127]:
df3 = pd.merge(merged, review, on = 'user_id', how = 'inner')

In [128]:
df3.head(2)

,title,id,user_id,playtime_forever,year,posted,item_id,recommend,sentiment_analysis
0,Carmageddon Max Pack,282010,UTNerd24,5,1997,"May 6, 2014.",244210,True,0
1,Carmageddon Max Pack,282010,UTNerd24,5,1997,"December 2, 2015.",440,True,0


Once our DataSet has all the columns we need, we proceed to save it.

In [ ]:
df3.to_parquet('./data/recommendations.parquet')

In [23]:
items = pd.read_parquet('./data/items.parquet')

#### We check how much memory is allocated in order to manipulate the 'items' DataFrame

In [54]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094082 entries, 0 to 5094081
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           string
 1   item_id           int32 
 2   item_name         string
 3   playtime_forever  int32 
dtypes: int32(2), string(2)
memory usage: 116.6 MB


#### There is a lot of memory waste since ``object`` ``dtype`` requieres more memory than an ``int32`` ``dtype``.

#### We change all of the DataFrames type, to have a better memory management

In [27]:
items['playtime_forever'] = items['playtime_forever'].astype('int32')
items['item_id'] = items['item_id'].astype('int32')
items['user_id'] = items['user_id'].astype('category')
items['item_name'] = items['item_name'].astype('category')

In [28]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094082 entries, 0 to 5094081
Data columns (total 4 columns):
 #   Column            Dtype   
---  ------            -----   
 0   user_id           category
 1   item_id           int32   
 2   item_name         category
 3   playtime_forever  int32   
dtypes: category(2), int32(2)
memory usage: 70.9 MB


# We gained 38.9 MB of memory

## We overwrite our previous file, which was larger.

In [29]:
items.to_parquet('./data/items.parquet')

### We proceed the same with the remaining DataFrames

In [62]:
recommend =pd.read_parquet('./data/recommendations.parquet')
recommend

,genres,year,user_id,playtime_forever,title
0,Action,1997,UTNerd24,5,Lost Summoner Kitty
1,Action,1997,I_DID_911_JUST_SAYING,0,Ironbound
2,Action,1997,76561197962104795,0,Real Pool 3D - Poolians
3,Action,1997,r3ap3r78,0,弹炸人2222
4,Action,1997,saint556,13,Battle Royale Trainer
...,...,...,...,...,...
9761456,Action,2004,76561198273508956,0,NaN
9761457,Action,2004,76561198282090798,0,NaN
9761458,Action,2004,943525,0,NaN
9761459,Action,2004,76561198283312749,9,NaN


In [37]:
recommend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9761461 entries, 0 to 9761460
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   genres            object
 1   year              int16 
 2   user_id           object
 3   playtime_forever  int32 
dtypes: int16(1), int32(1), object(2)
memory usage: 204.8+ MB


## There is a lot of memory waste in this DataFrame, for instance the `sentiment_analysis` column uses 64bits to store single-digit numbers

In [ ]:
recommend['playtime_forever'] = recommend['playtime_forever'].astype('int32')
recommend['year'] = recommend['year'].astype('int16')
recommend['item_id'] = recommend['item_id'].astype('int32')
recommend['recommend'] = recommend['recommend'].astype('bool')
recommend['sentiment_analysis'] = recommend['sentiment_analysis'].astype('int8')
recommend['user_id'] = recommend['user_id'].astype('category')


In [77]:
recommend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9761461 entries, 0 to 9761460
Data columns (total 7 columns):
 #   Column              Dtype   
---  ------              -----   
 0   genres              category
 1   year                int16   
 2   user_id             category
 3   playtime_forever    int32   
 4   title               category
 5   sentiment_analysis  int8    
 6   recommend           bool    
dtypes: bool(1), category(3), int16(1), int32(1), int8(1)
memory usage: 143.4 MB


## We saved 102 MB of memory

In [78]:
# Finally, delete unnecesary columns
# recommend.drop(['item_id', 'id', 'user_id', 'posted'], axis = 1, inplace=True)
recommend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9761461 entries, 0 to 9761460
Data columns (total 7 columns):
 #   Column              Dtype   
---  ------              -----   
 0   genres              category
 1   year                int16   
 2   user_id             category
 3   playtime_forever    int32   
 4   title               category
 5   sentiment_analysis  int8    
 6   recommend           bool    
dtypes: bool(1), category(3), int16(1), int32(1), int8(1)
memory usage: 143.4 MB


In [79]:
# We overwrite the file
recommend.to_parquet('./data/recommendations.parquet')

In [30]:
genre_functions = pd.read_parquet('./data/genre_functions.parquet')

### Inspect the DataFrame

In [40]:
genre_functions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9761461 entries, 0 to 9761460
Data columns (total 4 columns):
 #   Column            Dtype   
---  ------            -----   
 0   genres            category
 1   year              int16   
 2   user_id           category
 3   playtime_forever  int32   
dtypes: category(2), int16(1), int32(1)
memory usage: 104.9 MB


In [32]:
genre_functions.head()

,genres,year,user_id,playtime_forever
0,Action,1997,UTNerd24,5
1,Action,1997,I_DID_911_JUST_SAYING,0
2,Action,1997,76561197962104795,0
3,Action,1997,r3ap3r78,0
4,Action,1997,saint556,13


In [33]:
genre_functions['year'] = genre_functions['year'].astype('int16')
genre_functions['playtime_forever'] = genre_functions['playtime_forever'].astype('int32')
genre_functions['genres'] = genre_functions['genres'].astype('category')
genre_functions['user_id'] = genre_functions['user_id'].astype('category')

In [34]:
genre_functions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9761461 entries, 0 to 9761460
Data columns (total 4 columns):
 #   Column            Dtype   
---  ------            -----   
 0   genres            category
 1   year              int16   
 2   user_id           category
 3   playtime_forever  int32   
dtypes: category(2), int16(1), int32(1)
memory usage: 104.9 MB


In [35]:
genre_functions.to_parquet('./data/genre_functions.parquet')

## Create the smallest DataFrame possible, one for each function.

In [91]:
f1 = pd.read_parquet('./data/genre_functions.parquet')

In [92]:
f1 = f1.drop('user_id', axis=1)

In [93]:
f1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9761461 entries, 0 to 9761460
Data columns (total 3 columns):
 #   Column            Dtype   
---  ------            -----   
 0   genres            category
 1   year              int16   
 2   playtime_forever  int32   
dtypes: category(1), int16(1), int32(1)
memory usage: 65.2 MB


In [95]:
f1['playtime_forever']= f1['playtime_forever'].astype('int16')

In [98]:
f1.to_parquet('./data/f1.parquet')

In [99]:
f2 = pd.read_parquet('./data/genre_functions.parquet')

In [100]:
f2.to_parquet('./data/f2.parquet')

In [101]:
f3 = pd.read_parquet('./data/recommendations.parquet')

title,year,recommend,user_id,sentiment_analysis

In [ ]:
f3= f3.drop(['genres', 'playtime_forever'], axis=1, inplace=True)

In [115]:
f3

,year,user_id,title,sentiment_analysis,recommend
0,1997,UTNerd24,Lost Summoner Kitty,2,True
1,1997,I_DID_911_JUST_SAYING,Ironbound,2,True
2,1997,76561197962104795,Real Pool 3D - Poolians,2,True
3,1997,r3ap3r78,弹炸人2222,2,True
4,1997,saint556,Battle Royale Trainer,0,True
...,...,...,...,...,...
9761456,2004,76561198273508956,NaN,0,True
9761457,2004,76561198282090798,NaN,0,True
9761458,2004,943525,NaN,0,True
9761459,2004,76561198283312749,NaN,0,True


In [106]:
f3.to_parquet('./data/f3.parquet')

In [108]:
f4 = pd.read_parquet('./data/recommendations.parquet')

In [112]:
f4.drop(['genres', 'sentiment_analysis', 'playtime_forever', 'user_id'], axis = 1, inplace=True)

In [121]:
f4.to_parquet('./data/f4.parquet')

In [116]:
f5 = pd.read_parquet('./data/recommendations.parquet')

In [117]:
f5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9761461 entries, 0 to 9761460
Data columns (total 7 columns):
 #   Column              Dtype   
---  ------              -----   
 0   genres              category
 1   year                int16   
 2   user_id             category
 3   playtime_forever    int32   
 4   title               category
 5   sentiment_analysis  int8    
 6   recommend           bool    
dtypes: bool(1), category(3), int16(1), int32(1), int8(1)
memory usage: 143.4 MB


In [118]:
f5.drop(['genres', 'user_id', 'playtime_forever', 'title', 'recommend'], axis=1,inplace=True)

In [119]:
f5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9761461 entries, 0 to 9761460
Data columns (total 2 columns):
 #   Column              Dtype
---  ------              -----
 0   year                int16
 1   sentiment_analysis  int8 
dtypes: int16(1), int8(1)
memory usage: 27.9 MB


In [120]:
f5.to_parquet('./data/f5.parquet')

f1 = pd.read